# Gait analysis

In [ ]:
import os
from paradigma.gait.gait_analysis import extract_gait_features_io, detect_gait_io, extract_arm_activity_features_io, filter_gait_io
from paradigma.config import IMUConfig, GaitFeatureExtractionConfig, GaitDetectionConfig, ArmActivityFeatureExtractionConfig, FilteringGaitConfig
from paradigma.imu_preprocessing import preprocess_imu_data_io

In [ ]:
branch = 'gait'
sensor = 'imu'

path_to_data =  '../../../tests/data'
path_to_classifiers = os.path.join(path_to_data, '0.classification', branch)
path_to_sensor_data = os.path.join(path_to_data, '1.prepared_data', sensor)
path_to_preprocessed_data = os.path.join(path_to_data, '2.preprocessed_data', sensor)
path_to_extracted_features = os.path.join(path_to_data, '3.extracted_features', branch)
path_to_predictions = os.path.join(path_to_data, '4.predictions', branch)
path_to_quantification = os.path.join(path_to_data, '5.quantification', branch)

# Cell has the tag 'parameters', so it won't overwrite the parameters when running the notebook in tests

## Preprocessing

In [ ]:
config = IMUConfig()
preprocess_imu_data_io(path_to_sensor_data, path_to_preprocessed_data, config, sensor='both')

In [ ]:
from paradigma.imu_preprocessing import preprocess_imu_data
from paradigma.util import read_metadata
import tsdf

# Load data
config = IMUConfig()

metadata_time, metadata_values = read_metadata(str(path_to_sensor_data), str(config.meta_filename),
                                                str(config.time_filename), str(config.values_filename))
df = tsdf.load_dataframe_from_binaries([metadata_time, metadata_values], tsdf.constants.ConcatenationType.columns)

df = preprocess_imu_data(df, config, sensor='both')

In [ ]:
df

## Extract gait features

In [ ]:
config = GaitFeatureExtractionConfig()
extract_gait_features_io(path_to_preprocessed_data, path_to_extracted_features, config)

## Detect gait

In [ ]:
config = GaitDetectionConfig()
detect_gait_io(path_to_extracted_features, path_to_predictions, path_to_classifiers, config)

## Extract arm actvitiy features

In [ ]:
config = ArmActivityFeatureExtractionConfig()
extract_arm_activity_features_io(path_to_preprocessed_data, path_to_predictions, path_to_classifiers, path_to_extracted_features, config)

## Filter gait

In [ ]:
config = FilteringGaitConfig()
filter_gait_io(path_to_extracted_features, path_to_classifiers, path_to_predictions, config)

## Quantify arm swing

In [ ]:
# config = ArmSwingQuantificationConfig()
# quantify_arm_swing_io(path_to_extracted_features, path_to_predictions, path_to_classifiers, path_to_quantification, config)